# Remote access to Rubin data

- Tutorial : https://dp1.lsst.io/tutorials/api/api-101-2.html

- author Sylvie Dagoret-Campagne
- creation date : 2026-01-19

In [ ]:
import pyvo
import requests

In [ ]:
import os
token = os.environ["RSP_TOKEN"]

In [ ]:
token

## Create a persistent HTTP session and attach the access token using the Authorization header.

In [ ]:
session = requests.Session()
session.headers["Authorization"] = f"Bearer {token}"

## Connect to the Rubin TAP service

In [ ]:
rsp_tap_url = "https://data.lsst.cloud/api/tap"
service = pyvo.dal.TAPService(rsp_tap_url, session=session)

The TAP service object represents the remote catalog service and manages job submission and result retrieval.

## Define the ADQL query

Define an ADQL query that retrieves the visit table, as in DP1 Notebook tutorial https://dp1.lsst.io/tutorials/notebook/201/notebook-201-10.html

In [ ]:
query = """
SELECT visit, ra, dec, band, expMidptMJD
FROM dp1.Visit
WHERE CONTAINS(
      POINT('ICRS', ra, dec),
      CIRCLE('ICRS', 53.13, -28.10, 3)
    ) = 1
  AND band IN ('g', 'r')
ORDER BY expMidptMJD ASC
"""

## Submit and run the TAP job

Submit the query as an asynchronous TAP job and start execution.

In [ ]:
job = service.submit_job(query)
job.run()

In [ ]:
job.wait(phases=["COMPLETED", "ERROR"])
print("Job phase:", job.phase)

In [ ]:
if job.phase == "ERROR":
    job.raise_if_error()

## Retrieve and inspect the results

Fetch the query results, convert them to an Astropy table, and print the length of the table.

In [ ]:
results = job.fetch_result().to_table()
print(len(results))

In [ ]:
results[:50]

## Retrieve images

In [ ]:
from pyvo.dal.adhoc import DatalinkResults

#import lsst.afw.display as afwDisplay
#from lsst.afw.image import ExposureF
#from lsst.rsp import get_tap_service
#from lsst.rsp.utils import get_pyvo_auth

In [ ]:
target_ra = 53.076
target_dec = -28.110

In [ ]:
query = """SELECT * FROM ivoa.ObsCore
        WHERE CONTAINS(POINT('ICRS', {},{}), s_region) = 1
        AND dataproduct_subtype='lsst.visit_image'
        """.format(target_ra, target_dec)
print(query)

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
if job.phase == 'ERROR':
    job.raise_if_error()
assert job.phase == 'COMPLETED'
results = job.fetch_result().to_table()
print(len(results))

In [ ]:
results

In [ ]:
results['access_url'][0]

In [ ]:
datalink_url = results['access_url'][0]
dl_result = DatalinkResults.from_result_url(datalink_url, session=get_pyvo_auth())
image_url = dl_result.getrecord(0).get('access_url')